## Complete example for an autotrader that uses Predicto "My Picks"
You can use this notebook daily to submit yout latest Trade Picks that you have picked in (https://predic.to/autotrader) 


### For Predicto authentication
To use predicto api and reproduce this notebook, you'll need to have a valid Predicto account. 
If you don't have one, you can create one here: https://predic.to and get a free trial period.

To authenticate you'll need an api key. To retrieve it, login to https://predic.to and head to your [settings page](https://predic.to/account). Then paste it in the `predicto_api_key` variable below.

If you get any exception/error while running below code, please make sure your api key is correct and your subscription/trial is not expired.
Please note that there is a limit to the number of requests you can make per minute, depending on your account type.

### For Alpaca authentication
You'll need an alpaca.markets account. Then you can retrieve your API Key ID and Endpoint from your account page. You can use either paper money or real money. We recommend to experiment with a paper money account first.

More info about Alpaca trade api can be found here: https://github.com/alpacahq/alpaca-trade-api-python/

### Import needed packages

In [1]:
import sys
sys.path.append("../predicto_api/")

import pandas as pd

from predicto_api_wrapper import PredictoApiWrapper, TradeAction, TradeOrderType
from alpaca_api_wrapper import AlpacaApiWrapper

### Prepare and initialize our Alpaca API wrapper
You'll need to have a valid Alpaca account as mentioned above, and replace below variables with your own credentials.

In [2]:
alpaca_api_endpoint = 'https://paper-api.alpaca.markets' # use paper money endpoint for now (test env)
alpaca_api_key_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
alpaca_api_secret_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

alpaca_wrapper = AlpacaApiWrapper(alpaca_api_endpoint, alpaca_api_key_id, alpaca_api_secret_key)

### Prepare and initialize our Predicto wrapper
You'll need to have a valid Predicto account as mentioned above, and get an api key. Then replace the `predicto_api_key` variable below

In [10]:
predicto_api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
predicto_api_wrapper = PredictoApiWrapper(predicto_api_key)
predicto_api_wrapper.set_alpaca_api_wrapper(alpaca_wrapper)

### Make sure our alpaca keys work as expected
Retrieve some latest prices

In [4]:
print('Latest price of AAPL is: ', alpaca_wrapper.get_latest_price('AAPL'))
print('Latest price of FB   is: ', alpaca_wrapper.get_latest_price('FB'))
print('Latest price of MSFT is: ', alpaca_wrapper.get_latest_price('MSFT'))
print('Latest price of NIO  is: ', alpaca_wrapper.get_latest_price('NIO'))
print('Latest price of TSLA is: ', alpaca_wrapper.get_latest_price('TSLA'))
print('Latest price of V    is: ', alpaca_wrapper.get_latest_price('V'))

print('Looks good \N{grinning face}')

Latest price of AAPL is:  123.94
Latest price of FB   is:  268.82
Latest price of MSFT is:  217.53
Latest price of NIO  is:  47.4
Latest price of TSLA is:  660.22
Latest price of V    is:  205.27
Looks good 😀


### Let's make sure our Predicto Key works as expected
Retrieve supported stocks 

In [5]:
# Get all supported stocks
stocks_json = predicto_api_wrapper.get_supported_tickers()
stocks_df = pd.DataFrame(stocks_json)

# print some information
print('Total tickers supported: {0}'.format(len(stocks_df)))
print('Here is a sample:')
stocks_df.head(15)

Total tickers supported: 44
Here is a sample:


,RelatedStock,RelatedCompany
0,AMD,AMD
1,ATVI,Activision
2,BABA,Alibaba
3,AMZN,Amazon
4,AAL,American Airlines
5,AAPL,Apple
6,BIDU,Baidu
7,BAC,Bank of America
8,BKNG,Booking Holdings
9,CVX,Chevron


# We are ready! 
# Let's run our autotrader using the Trade Picks we selected at Predicto website!
Below call will get your latest Predicto Trade Picks and will submit them all!

## You can run this daily just after market open (9.31am E.T.)
Feel free to re-adjust below parameters to match your criteria

    investment_per_trade : how much money to use per trade (note we'll submit an order for as many stocks as possible up to this number. If it's not enough for a single stock we'll skip)
    trade_order_type     : TradeOrderType.Bracket or TradeOrderType.TrailingStop. Bracket will set fixed stop loss and take profit prices. TrailingStop will set a trailing stop price.

In [9]:
predicto_api_wrapper.submit_my_latest_trade_picks(
    investment_per_trade=1000,
    trade_order_type=TradeOrderType.Bracket)

Processing Predicto "My" Trade Picks from 2020-12-20
- More details https://predic.to/exploreroi?my=1&sdate=2020-12-20

Processing 1/2, Symbol: V
	 Expected change : -1.17% !
	 Trade Action    : Sell !
	 Avg Uncertainty : 4.65% !
	 Avg ROI of model: 0.59% !

> Alpaca: Submitting Bracket order...
------------------------
Wont process - Price already moved: action == TradeAction.Sell and (latestPrice <= targetSellPrice)
Error: validateLatestPriceAndStopLoss failed
------------------------

Processing 2/2, Symbol: SNAP
	 Expected change : 1.08% !
	 Trade Action    : Buy !
	 Avg Uncertainty : 2.50% !
	 Avg ROI of model: 0.67% !

> Alpaca: Submitting Bracket order...
------------------------
Expected price         : 52.99
Actual   price         : 52.18
Original stopLossPrice : 52.8469317397
New stopLossPrice      : 52.0391186672494
Target Price           : 53.562273041
Success: submit_order ID: 8776cd8c-b116-4aba-9ff8-0d8c8cf8362f
------------------------

Submitted 1 hedged orders: ['SNAP'

# We  submitted our Personal hand picked Trade Picks !
## Keep in mind that Alpaca doesn't allow short selling for some stocks, so you might get a relevant error.
Those orders will be closed once their take-profit or stop-loss price we set are hit, depending on the TradeOrderType used.

You can inspect your oders at https://app.alpaca.markets/paper/dashboard/orders

You can schedule this script to run daily before market open.

Make sure you understand the risks if you are using real money!